Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

### Инструкция по выполнению проекта

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# 1. Подготовка

*Проект делал через colab, чтобы ускорить вычисления, используя GNU через cuda. Для проверки своего кода и для упрощения ревью взял для BERTa часть датасета, буду прикладывать скриншоты выполнения и метрик.*

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#!pip install transformers

     |████████████████████████████████| 1.1MB 8.8MB/s 
     |████████████████████████████████| 890kB 31.0MB/s 
     |████████████████████████████████| 1.1MB 52.3MB/s 
     |████████████████████████████████| 3.0MB 53.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=5ed0df17d6ac9e308eabb6888c905f6ac8c31a9e5cf9afba3c5928d97dc44c2c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
pip install thinc-gpu-ops

     |████████████████████████████████| 491kB 8.6MB/s 
  Created wheel for thinc-gpu-ops: filename=thinc_gpu_ops-0.0.4-cp36-cp36m-linux_x86_64.whl size=221838 sha256=4bb4506677684eb3c19b4ea0566559b724804a6cf0bea19f1170d553ad1a0e42
  Stored in directory: /root/.cache/pip/wheels/eb/ba/a3/9af9f326ed0d75a4540378af64a05a0e42be39d9b8513f3aea
Successfully built thinc-gpu-ops


In [4]:
import re
import torch
import transformers
import pandas as pd
import numpy as np
import csv
from tqdm import notebook
from tqdm._tqdm_notebook import tqdm_notebook
import nltk
import spacy
import thinc_gpu_ops
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer
import tensorflow as tf
from IPython.display import Image
from IPython.core.display import HTML 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  


In [7]:
#thinc_gpu_ops.AVAILABLE
#spacy.require_gpu() лемматизация spacy на GPU вышла медленнее

In [8]:
#data_comments = pd.read_csv('/content/drive/My Drive/Colab Notebooks/toxic_comments.csv')
data_comments = pd.read_csv('/datasets/toxic_comments.csv')
#data_comments = data_comments.sample(500).reset_index(drop=True)

In [9]:
data_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [10]:
data_comments.shape

(159571, 2)

In [11]:
data_comments[:5]

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


*Твиты на английском, будем использовать другие библиотеки, чем на тренажере*

In [12]:
data_comments['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

## Подготовка данных, для рассчета TF-IDF

*Убираем ненужное, используя регулярные выражения*

In [13]:
def clean_text(text):
    # filter to allow only alphabets
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    
    text = text.split()
    #text = text.lower()
    
    return " ".join(text)

In [14]:
tqdm_notebook.pandas()
corpus_clean =  data_comments.text.progress_apply(clean_text)

In [15]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

*Лемматизируем с помощью spacy*

In [16]:
def lemmatize(text):
    #spacy.prefer_gpu()
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

In [17]:
corpus_lemm = [lemmatize(x) for x in tqdm_notebook(corpus_clean)]

---

In [18]:
features = corpus_lemm
target = data_comments['toxic']

In [19]:
features_train, features_test, target_train, target_test = (
train_test_split(features, target, test_size = .1, random_state = 12345))

*Скачиваем стоп-слова для рассчета TF-IDF*

In [20]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [21]:
del features,target, corpus_lemm

*Далее получать TF-IDF TfidfVectorizer будем в рамках пайплайна вместе с обучением модели*

---

## BERT

*Для рассчета эмбедингов по всему датасету воспользуемся возможностями GPU*

In [37]:
#device_name = tf.test.gpu_device_name()
#The device name should look like the following:
#if device_name == '/device:GPU:0':
#    print('Found GPU at: {}'.format(device_name))
#else:
#    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [38]:
Image(url = 'https://i.ibb.co/YLTjkMx/2020-10-03-22-32-46.jpg')

In [39]:
# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [40]:
Image(url='https://i.ibb.co/sVwVCXJ/2020-10-03-22-33-02.jpg')

*Загружаем данные для BERT*

In [41]:
model_class, tokenizer_class, pretrained_weights = transformers.BertModel, transformers.BertTokenizer, 'bert-base-uncased'
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

*Разбиваем твиты на токены + специальные токены начало/конец (обрезал датасет для демонстрации расчетов). Пробовал рассчитать BERT по уже лемматизированным данным, метрика вышла хуже.*

In [42]:
tokenized = [tokenizer.encode(x[:510], add_special_tokens=True) for x in tqdm_notebook(corpus_clean[:400])]

*Делаем тексты одинаковой длины*

In [43]:
padded = np.array([i + [0]*(512 - len(i)) for i in tokenized])

*Нули не имеют значимости*

In [44]:
attention_mask = np.where(padded != 0, 1, 0)

In [45]:
if torch.cuda.is_available():
  model.cuda()
  model.to(device)

*Производим рассчет эмбеддингов по каждому батчу с размером 10 и собираем эмбеддинги вместе*

In [46]:
batch_size = 10
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
    batch = torch.LongTensor(np.array(padded[batch_size*i:batch_size*(i+1)])).to(device)
    attention_mask_batch =\
    torch.LongTensor(np.array(attention_mask[batch_size*i:batch_size*(i+1)])).to(device)
    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
    #with open('embeddings.csv', 'a') as fp:
        #for row in batch_embeddings[0][:,0,:].numpy():
            #csv.writer(fp, delimiter=",").writerow(row)

    embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

In [47]:
Image(url='https://i.ibb.co/RQRCt1q/2020-10-03-22-47-39.jpg')

In [51]:
features_rubert = np.concatenate(embeddings)
target_rubert = data_comments['toxic'][:400]#[:-1] в датасете 159 571 строки, одну отбросил из-за батчей;)

In [52]:
features_rubert.shape

(400, 768)

In [53]:
features_train_b, features_test_b, target_train_b, target_test_b = (
train_test_split(features_rubert, target_rubert, test_size = .1, random_state = 12345))

In [54]:
del embeddings, features_rubert, target_rubert, padded, tokenized, attention_mask, data_comments

# 2. Обучение

## ***TfidfVectorizer***

*LogisticRegression*

In [55]:
pipe = make_pipeline(
    TfidfVectorizer(stop_words = stopwords), 
    LogisticRegression(max_iter=1000, class_weight = 'balanced', solver = 'lbfgs'))

In [56]:
params = {'tfidfvectorizer__ngram_range' : [(1,2)],
          'logisticregression__C' : [1,10]}

In [57]:
grid_tfidf = GridSearchCV(pipe, params, cv=3, scoring = make_scorer(f1_score)) 

In [58]:
grid_tfidf.fit(features_train, target_train)

GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('tfidfvectorizer',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                               

In [59]:
grid_tfidf.best_params_

{'logisticregression__C': 10, 'tfidfvectorizer__ngram_range': (1, 2)}

In [60]:
grid_tfidf.best_score_

0.7830019658317852

In [61]:
tfidf_test = grid_tfidf.score(features_test,target_test)

In [62]:
tfidf_test

0.7813138686131388

---

*RidgeClassifier*

In [63]:
pipe = make_pipeline(
    TfidfVectorizer(stop_words = stopwords), RidgeClassifier(max_iter=1000))

In [64]:
params = {'tfidfvectorizer__ngram_range' : [(1,2)]}

In [65]:
grid_tfidf_r = GridSearchCV(pipe, params, cv=3, scoring = make_scorer(f1_score)) 

In [66]:
grid_tfidf_r.fit(features_train, target_train)

GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('tfidfvectorizer',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                               

In [67]:
grid_tfidf_r.best_params_

{'tfidfvectorizer__ngram_range': (1, 2)}

In [68]:
grid_tfidf_r.best_score_

0.736808443945292

In [69]:
tfidf_r_test = grid_tfidf_r.score(features_test,target_test)

In [70]:
tfidf_r_test

0.7378157239416578

*Лучшие результаты показала логистическая регрессия*

---

## *BERT*

*LogisticRegression*

In [71]:
log_model = LogisticRegression(max_iter=1000, solver = 'lbfgs')

params = {'C' : [1,10], 'class_weight': ['balanced']}

rubert_grid = GridSearchCV(log_model,param_grid = params,
                           scoring = make_scorer(f1_score), cv = 3)

In [72]:
rubert_grid.fit(features_train_b, target_train_b)

GridSearchCV(cv=3, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=1000, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1, 10], 'class_weight': ['balanced']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=make_scorer(f1_score), verbose=0)

In [73]:
rubert_grid.best_params_

{'C': 1, 'class_weight': 'balanced'}

In [74]:
rubert_grid.best_score_

0.7454643048845947

In [75]:
Image(url = 'https://i.ibb.co/0JNJvfq/2020-10-05-07-12-34.jpg" alt="2020-10-05-07-12-34')

In [76]:
rubert_test = rubert_grid.score(features_test_b, target_test_b)

---
*RidgeClassifier*

In [79]:
ridge_model = RidgeClassifier(max_iter=1000, solver = 'auto')

params = {'class_weight': ['balanced']}

rubert_grid_r = GridSearchCV(ridge_model,param_grid = params,
                           scoring = make_scorer(f1_score), cv = 3)

In [80]:
rubert_grid_r.fit(features_train_b, target_train_b)

GridSearchCV(cv=3, error_score=nan,
             estimator=RidgeClassifier(alpha=1.0, class_weight=None,
                                       copy_X=True, fit_intercept=True,
                                       max_iter=1000, normalize=False,
                                       random_state=None, solver='auto',
                                       tol=0.001),
             iid='deprecated', n_jobs=None,
             param_grid={'class_weight': ['balanced']}, pre_dispatch='2*n_jobs',
             refit=True, return_train_score=False,
             scoring=make_scorer(f1_score), verbose=0)

In [81]:
rubert_grid_r.best_params_

{'class_weight': 'balanced'}

In [82]:
rubert_grid_r.best_score_

0.6031746031746031

In [83]:
Image(url = 'https://i.ibb.co/d2hnYt9/2020-10-05-07-13-03.jpg" alt="2020-10-05-07-13-03')

In [84]:
rubert_r_test = rubert_grid_r.score(features_test_b, target_test_b)

In [85]:
rubert_r_test

0.7499999999999999

In [86]:
Image(url = 'https://i.ibb.co/zPQjT75/2020-10-05-07-13-13.jpg" alt="2020-10-05-07-13-13')

*Лучший score опять у логистической регрессии*

# 3. Выводы

*Среди протестированных моделей обучения лучшую метрику получила Логистическая регрессия*

In [87]:
pd.DataFrame(data = [[grid_tfidf.best_score_, tfidf_test, rubert_grid.best_score_, rubert_test]],
             columns=['TF-IDF_train', 'TF-IDF_test', 'BERT_train', 'BERT_test'])

,TF-IDF_train,TF-IDF_test,BERT_train,BERT_test
0,0.783002,0.781314,0.745464,0.666667


In [88]:
Image(url = 'https://i.ibb.co/84ZcwGY/2020-10-05-07-11-41.jpg" alt="2020-10-05-07-11-41')

Опробованы следующие способы обработки текстовых данных :

*   BERT
*   TfidfVectorizer

*Логистическая регрессия + TF-IDF + spacy(лемматизация) + "использование регулярных выражений" -  выполняет условие значения метрики F1, указанного в задании. Учитывая время на подготовку данных и время обучения и специфику данных - стоит использовать данный пайплайн для классификации сообщений на токсичность.*

# Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны